# Generate DFS

Este notebook tem como função gerar os dataframes que serão utilizados no desenvolvimento do projeto, para tanto, teremos três data frames principais:
* Dados transacionais: dados de cada usuário em um range de tempo específico, ou seja, quais músicas e genero aquele usuário escutou (optei por ultilizar os primeiros seis meses, delineando o estudo como sazonal semestral)
* Dados informacionais do genero das músicas
* Dados dos artistas

### Requirements

In [2]:
import random
import numpy as np
import pandas as pd
from datetime import datetime

### Functions

## transactional data

In [3]:
def create_df_modeling(start_date, end_date,
                       low_user, high_user, dau,
                       low_track, high_track, 
                       min_plays, max_plays,
                       min_n_tracks, max_n_tracks
                      ):
    # creates a range of dates
    date_range = pd.date_range(start_date, end_date)
    # creates empty array for the final user array
    user_array_acum = []
    # creates empty array for the final id_date array
    date_array_acum = []
    # creates empty array for the final track array
    tracks_array_acum = []
    # creates empty array for the final plays array
    plays_array_acum = []
    # minimum number of random listeners per day (dau)
    length_user = 1 + int((high_user-low_user) * dau)
    # for loop to random create transactional data
    for current_date in date_range:
        #
        user_array = np.random.randint(low_user, high_user, size = length_user)
        user_array_col = np.reshape(user_array, (len(user_array), 1))
        user_array_acum.append(user_array_col)
        #
        date_array = current_date.strftime("%Y-%m-%d")
        date_array = np.full(len(user_array), current_date)
        date_array_col = np.reshape(date_array, (len(date_array), 1))
        date_array_acum.append(date_array_col)
    # for loop that ranges from the min id_user to the max id_user
    for k in range(0,len(np.concatenate(user_array_acum))):
        # defines randomly the maximum range of tracks a listener can listen in 1 single day
        length_size = random.randint(min_n_tracks, max_n_tracks)
        # selects at random an array of id_track for the specific listener
        tracks_array = np.random.randint(low_track, high_track, size = length_size)
        tracks_array_acum.append(tracks_array)
        # select at random the average of plays given by each user on a giving day
        plays_array = np.random.randint(min_plays, max_plays, size = length_size)
        plays_array_acum.append(plays_array) 
    # concatenates the arrays of date, id_user and plays on the final dataframe
    date_array_final = np.concatenate(date_array_acum, axis = 0)
    user_array_final = np.concatenate(user_array_acum, axis = 0)
    # creates the final transactional dataframe with id_users and number of plays for each day
    df = pd.DataFrame({'id_date':date_array_final.reshape(len(date_array_final)),
                       'user_id':user_array_final.reshape(len(user_array_final)),
                       'id_tracks': tracks_array_acum,
                       'plays':plays_array_acum
                      })
    # explodes the final dataframe to create a vertical transactional table
    df = df.explode(['id_tracks','plays']).reset_index(drop=True)
    #
    return df

## Dimensional content

In [4]:
def create_dim_content(n_tracks, min_tracks, max_tracks, n_artists, n_genres):
    # creates an initial list of id_tracks
    id_track_list = [i for i in range(1,n_tracks+1)]
    # creates an initial list of id_artists
    id_artist_list = [i for i in range(1, n_artists+1)]
    # creates an initial list of id_genres
    id_genres = [i for i in range(1,n_genres)]
    # creates array with the specific size of tracks by artists to be distributed amongst all artists
    artist_array_sizes = np.random.randint(min_tracks, max_tracks, size = n_artists).tolist()
    # Create a copy of the list
    list_copy = id_track_list.copy()
    # initiates an empty array to store results of for loop append (arrays of id_tracks)
    sampled_arrays_acum = []
    # initiates an empty array to store results of for loop append (arrays of id_genre)
    sampled_arrays_genre_acum = []
    # for loop to interate and create the dimensional data for id_artists and id_tracks
    for size in artist_array_sizes:
        if size <= len(list_copy):
            sampled_arrays = random.sample(list_copy, size)
            sampled_arrays_genre = random.choices(id_genres, k = size)
            sampled_arrays_acum.append(sampled_arrays)
            sampled_arrays_genre_acum.append(sampled_arrays_genre)
            list_copy = [item for item in list_copy if item not in sampled_arrays]
    # creates the final dimensional dataframe
    df = pd.DataFrame({'id_artist':id_artist_list,
                       'id_tracks':sampled_arrays_acum,
                       'id_genre':sampled_arrays_genre_acum
                      })
    # explodes the final dataframe to create a vertical dimensional table
    df = df.explode(['id_tracks','id_genre']).reset_index(drop = True)
    #
    return df

## Features for artists

In [5]:
def creates_features(n_features,n_artists,std_max, min_mean, max_mean):
    # creates an array with standard deviation of a uniform distribution
    std_array = [random.uniform(0, std_max) for i in range(0,n_features)]
    # creates an array with mean values of a uniform distribution
    mean_array = [random.randint(min_mean, max_mean) for i in range(0,n_features)]
    # creates an empty array with the number of rows as the same of the length of the quantity of artists
    result = np.empty((n_artists, 0))
    for i in range(0,n_features):
        feature_i = np.random.normal(mean_array[i], std_array[i] * mean_array[i], n_artists)
        feature_i = feature_i.astype(int)
        result = np.column_stack((result, feature_i))
        result = result.astype(int)
    # creates the list of id_artist
    id_artist = [i for i in range(1,n_artists+1)]
    # fills values on the result array
    result = np.column_stack((id_artist,result))
    # creates a list with only one element
    artist_col_names = ['id_artist']
    # creates a list with the names of the features that will be used on the final dataframe
    feat_col_names = ['Feature'+str(i) for i in range(1,n_features+1)]
    # adds two lists (elements will be use as the header of the final dataframe)
    col_names = artist_col_names + feat_col_names
    # final dataframe with features by artist
    df = pd.DataFrame(result, columns = col_names)
    return df

# Pipeline

## Defining global parameters

In [12]:
datetime(2023, 6, 30).strftime('%Y-%m-%d')

'2023-06-30'

In [23]:
########################################################################################################
######################### input parameters for the create_df_modeling function #########################
# Define the range of dates for the trasactional dataset
start_date = datetime(2023, 1, 1).strftime('%Y-%m-%d')
end_date = datetime(2023, 7, 30).strftime('%Y-%m-%d')
# defines the minimum id_user to consider
low_user = 1
# defines the maximum id_user to consider
high_user = 50
# defines the daily audience (the percentage of the sample size of users) for each id_date
dau = 0.1
# defines the minimum id_track available
low_track = 1
# defines the maximum id_track available
high_track = 4000
# defines the minimum number of plays a listener can hit play every day
min_plays = 1
# defines the maximum number of plays a listener can hit play every day
max_plays = 20
# defines the minimum number of tracks a listener can listen to every day
min_n_tracks = 2
# defines the maximum number of tracks a listener can listen to every day
max_n_tracks = 5


########################################################################################################
################ input parameters for the create_dim_content function ##################################
# defines the total number of tracks
n_tracks = high_track
# defines the total number of id_artist
n_artists = 500
# defines the total number of genres
n_genres = 5
# defines the minimum number of tracks by id_artist
min_tracks = 2 * round((n_tracks/n_artists),0) * 0.2
# defines the maximum number of tracks by id_artist
max_tracks = 2 * round((n_tracks/n_artists),0) * 0.8


########################################################################################################
################ input parameters for the creates_features function ##################################
# number of features that are going to appear on the final dataframe
n_features = 10
# maximum standard deviation value for each of the feature
std_max = 1
# minimum value for the mean value for each of the features
min_mean = 100
# maximum value for the mean value for each of the features
max_mean = 1500

## Creates df transactional data

In [24]:
#
df = create_df_modeling(start_date, end_date,
                        low_user, high_user, dau,
                        low_track, high_track, 
                        min_plays, max_plays,
                        min_n_tracks, max_n_tracks
                       )

## Creates df dimensional content

In [25]:
dc = create_dim_content(n_tracks,min_tracks,max_tracks,n_artists, n_genres)

## Creates df with features by artist

In [26]:
df_feat = creates_features(n_features,n_artists,std_max, min_mean, max_mean)

# Artefacts

## Transactional data source (reference modeling dataframe)

In [27]:
df

,id_date,user_id,id_tracks,plays
0,2023-01-01,41,611,3
1,2023-01-01,41,128,10
2,2023-01-01,41,478,12
3,2023-01-01,1,3003,17
4,2023-01-01,1,1778,13
...,...,...,...,...
3714,2023-07-30,30,1624,5
3715,2023-07-30,30,1981,2
3716,2023-07-30,30,1774,3
3717,2023-07-30,30,1973,17


In [35]:
df.dtypes

id_date      datetime64[ns]
user_id               int32
id_tracks            object
plays                object
holiday               int32
dtype: object

In [36]:
df.isnull().sum()

id_date      0
user_id      0
id_tracks    0
plays        0
holiday      0
dtype: int64

In [30]:
# identificando feriados
from datetime import datetime, date
import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, GoodFriday, Holiday, Easter, Day

class Feriados(AbstractHolidayCalendar):
    rules = [
        Holiday('Confraternização Universal', month=1, day=1),
        Holiday('Aniversário de São Paulo', month=1, day=25),
        Holiday('Segunda-Feira de Carnaval', month=1, day=1, offset=[Easter(), Day(-48)]),
        Holiday('Terça-Feira de Carnaval', month=1, day=1, offset=[Easter(), Day(-47)]),
        Holiday('Quarta-Feira de Cinzas', month=1, day=1, offset=[Easter(), Day(-46)]),
        # Sexta-feira Santa
        GoodFriday,
        Holiday('Corpus Christi', month=1, day=1, offset=[Easter(), Day(60)]),
        Holiday('Tiradentes', month = 4, day = 21),
        Holiday('Dia do Trabalho', month = 5, day = 1),
        Holiday('Revolução Constitucionalista', month=7, day=9, start_date='1997-01-01'),
        Holiday('Independência do Brasil', month = 9, day = 7),
        Holiday('Nossa Senhora Aparecida', month = 10, day = 12),
        Holiday('Finados', month = 11, day = 2),
        Holiday('Proclamação da República', month = 11, day = 15),
        Holiday('Dia da Consciencia Negra', month=11, day=20, start_date='2004-01-01'),
        Holiday('Vespera de Natal', month=12, day=24),
        Holiday('Natal', month = 12, day = 25)]

sp_cal = Feriados()
sp_feriados = pd.offsets.CustomBusinessDay(calendar=sp_cal)
feriados_sp = sp_cal.holidays(datetime(2000, 12, 31), datetime(2079, 12, 31))

start = datetime(2023, 1, 1)
end = datetime(2023, 7, 30)
date_fds = pd.bdate_range(start, end, freq='C', holidays=feriados_sp)

In [49]:
df['holiday'] = np.where(df.id_date.isin([date_fds]), 0, 1)
df['plays'] = df.plays.astype(int)

In [50]:
df.describe()

,id_date,user_id,plays,holiday
count,3719,3719.000000,3719.000000,3719.0
mean,2023-04-16 11:59:01.919870976,24.397956,10.021780,1.0
min,2023-01-01 00:00:00,1.000000,1.000000,1.0
25%,2023-02-24 00:00:00,12.000000,5.000000,1.0
50%,2023-04-18 00:00:00,24.000000,10.000000,1.0
75%,2023-06-07 00:00:00,37.000000,15.000000,1.0
max,2023-07-30 00:00:00,49.000000,19.000000,1.0
std,NaN,14.429244,5.479122,0.0


In [48]:
df.user_id.nunique()

49

## Dimensional data source with musical genre for each track

In [34]:
dc

,id_artist,id_tracks,id_genre
0,1,1380,1
1,1,460,4
2,1,1201,2
3,1,3053,2
4,2,3142,3
...,...,...,...
3408,499,3265,3
3409,499,3626,2
3410,500,938,4
3411,500,1952,1


In [37]:
dc.dtypes

id_artist     int64
id_tracks    object
id_genre     object
dtype: object

In [38]:
dc.isnull().sum()

id_artist    0
id_tracks    0
id_genre     0
dtype: int64

In [43]:
dc.id_artist.nunique()

500

In [44]:
dc.id_tracks.nunique()

3413

In [45]:
dc.id_genre.nunique()

4

## Data source with feature values for each artist

In [19]:
df_feat

,id_artist,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10
0,1,1397,1073,1303,2338,1762,164,2506,121,750,52
1,2,890,670,1462,-687,2700,113,1610,128,694,182
2,3,1299,400,1094,448,1402,143,478,802,768,155
3,4,797,705,1619,-688,560,55,2843,242,187,51
4,5,1344,713,1039,4246,1915,29,878,1079,417,25
...,...,...,...,...,...,...,...,...,...,...,...
495,496,1228,522,1217,-109,568,174,1375,207,761,219
496,497,1205,348,1665,1377,1683,193,1670,81,425,7
497,498,1386,512,1573,757,869,99,1478,373,332,87
498,499,1225,512,905,1409,1009,66,891,662,418,48


In [39]:
df_feat.dtypes

id_artist    int32
Feature1     int32
Feature2     int32
Feature3     int32
Feature4     int32
Feature5     int32
Feature6     int32
Feature7     int32
Feature8     int32
Feature9     int32
Feature10    int32
dtype: object

In [40]:
df_feat.isnull().sum()

id_artist    0
Feature1     0
Feature2     0
Feature3     0
Feature4     0
Feature5     0
Feature6     0
Feature7     0
Feature8     0
Feature9     0
Feature10    0
dtype: int64

## Juntando as bases

In [55]:
dc

,id_artist,id_tracks,id_genre
0,1,1380,1
1,1,460,4
2,1,1201,2
3,1,3053,2
4,2,3142,3
...,...,...,...
3408,499,3265,3
3409,499,3626,2
3410,500,938,4
3411,500,1952,1


In [60]:
df_all = df.merge(dc, how='left').merge(df_feat, how = 'left')

In [63]:
df_all.to_parquet(r'raw_data/raw_recommendation_data.parquet')

## Conclusões

Sobre as features:
artistas unicos
user unicos
tracks unicas

Entre a base de df e dc precisarei completar


Sobre o delineamento do problema: Ultilizei um periodo de sazonalidade semestral neste primeiro momento, porém como proximas evoluções do modelo, seria interessante desenvolver um modelo de recomendação para cada época do ano, entendendo que a temperatura média e a estação do ano pode influenciar na nmúsica a ser ouvida. Uma analogia são músicas de natal no proprio natal.
Além disso, seria interessante também explorar um modelo para dias uteis e não uteis, com a hipotese de que em dias uteis ouvimos mais músicas de foco para trabalhar e dias não uteis músicas de descansar.

Com isso, juntei as bases e passamos a proxima etapa, resolução de dados faltantes e festure engineering das variáveis.